## Code for original data preprocessing from Helena Ma's raw data 
Run this code to process allexps.csv to create folders for each dataset of each plasmid+inhibitor combination for the antibiotic treated dataset

In [1]:
import os
# Import basic packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



In [2]:
#import full csv file
df=pd.read_csv('allexps.csv',dtype={'plasmid_name':object})

In [3]:
'''create a dataframe for a specific dataset
- run this chunk with different combinations of plasmid and inhibitor for different datasets'''
#create mask for the conditions I want
plasmid='Bla'
inhibitor='SUL'
mask=(df['mixed_flag']=='Y')&(df['plasmid_name']==plasmid)&(df['inhibitor_name']==inhibitor)

# Create the folder to contain this set of data
folder_name=f'Data files/{plasmid}_{inhibitor}'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f"Folder '{folder_name}' created successfully!")
else:
    print(f"Folder '{folder_name}' already exists.")

#create dataframe of only the masked conditions
df_int=df[mask]
#create dataframe for condition information
conditions=df_int.iloc[:,:8].drop_duplicates(ignore_index=True)
#create dataframes of timecourses
time_vector=None
for index, row in conditions.iterrows():
    condition_mask=(df_int.iloc[:,:8]==row).all(axis=1)
    if time_vector is None:
        time_vector=np.array(df_int[condition_mask]['Hours'])
        od=np.empty((0,len(time_vector)))
        gfp=np.empty((0,len(time_vector)))
        bfp=np.empty((0,len(time_vector)))
    od=np.vstack((od,np.array(df_int[condition_mask]['OD600'])))
    gfp=np.vstack((gfp,np.array(df_int[condition_mask]['GFP'])))
    bfp=np.vstack((bfp,np.array(df_int[condition_mask]['BFP'])))
#save the time vector and plate reading array files
np.save(f'Data files/{plasmid}_{inhibitor}/{plasmid}_{inhibitor}_time.npy',time_vector)
np.save(f'Data files/{plasmid}_{inhibitor}/{plasmid}_{inhibitor}_od.npy',od)
np.save(f'Data files/{plasmid}_{inhibitor}/{plasmid}_{inhibitor}_gfp.npy',gfp)
np.save(f'Data files/{plasmid}_{inhibitor}/{plasmid}_{inhibitor}_bfp.npy',bfp)
conditions.to_csv(f'Data files/{plasmid}_{inhibitor}/{plasmid}_{inhibitor}_conditions.csv',index=False)
#select od values 0:145 from od600 column transpose and keep concatenating vertically

Folder 'Data files/Bla_SUL' already exists.


In [7]:
'no need to re-run this - unique_rows already saved to excel'
#this code works to create an excel file of the unique conditions from all of Helena's data
#create dataframe of just mixed populations
mix_df=df[df['mixed_flag']=='Y']
#create dataframe of just the unique conditions (eliminating the full time course)
unique_rows=mix_df.iloc[:,:8].drop_duplicates()
#save the unique conditions to excel for viewing
#unique_rows.to_excel('unique_rows.xlsx',index=False)